# Part 1 - Performing Tritium Breeding Ratio (TBR) simulations

This python notebook allows users to perform neutronics simulations to tally tritium production.

In [ ]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/Vc7Qy7QW4o8" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

This example uses a simple sphere model with a breeder material and a tritium production tally.

This first code block makes the geometry, materials and settings for the neutronics model.

In [ ]:
import openmc

# MATERIALS

breeder_material = openmc.Material()  # lithium lead chemical equation is Pb84.2Li15.8
breeder_material.add_element('Pb', 84.2, percent_type='ao')
# 50% enriched lithium 6, natural percentage of lithium 6 is just 7% 
breeder_material.add_element('Li', 15.8, percent_type='ao', enrichment=50.0, enrichment_target='Li6', enrichment_type='ao')
breeder_material.set_density('atom/b-cm', 3.2720171e-2)  # around 11 g/cm3


steel = openmc.Material()
steel.set_density('g/cm3', 7.75)
steel.add_element('Fe', 0.95, percent_type='wo')
steel.add_element('C', 0.05, percent_type='wo')

my_materials = openmc.Materials([breeder_material, steel])


# GEOMETRY

# surfaces
vessel_inner = openmc.Sphere(r=500)
first_wall_outer_surface = openmc.Sphere(r=510)
breeder_blanket_outer_surface = openmc.Sphere(r=610, boundary_type='vacuum')


# cells
inner_vessel_region = -vessel_inner
inner_vessel_cell = openmc.Cell(region=inner_vessel_region)

first_wall_region = -first_wall_outer_surface & +vessel_inner
first_wall_cell = openmc.Cell(region=first_wall_region)
first_wall_cell.fill = steel

breeder_blanket_region = +first_wall_outer_surface & -breeder_blanket_outer_surface
breeder_blanket_cell = openmc.Cell(region=breeder_blanket_region)
breeder_blanket_cell.fill = breeder_material

my_geometry = openmc.Geometry([inner_vessel_cell, first_wall_cell, breeder_blanket_cell])


# SIMULATION SETTINGS

# Instantiate a Settings object
my_settings = openmc.Settings()
my_settings.batches = 10
my_settings.inactive = 0
my_settings.particles = 500
my_settings.run_mode = 'fixed source'

# Create a DT point source
my_source = openmc.Source()
my_source.space = openmc.stats.Point((0, 0, 0))
my_source.angle = openmc.stats.Isotropic()
my_source.energy = openmc.stats.Discrete([14e6], [1])
my_settings.source = my_source




# added a cell tally for tritium production
cell_filter = openmc.CellFilter(breeder_blanket_cell)
tbr_tally = openmc.Tally(name='TBR')
tbr_tally.filters = [cell_filter]
tbr_tally.scores = ['(n,Xt)']  # Where X is a wildcard character, this catches any tritium production
# this allows the tally to be recorded per nuclide so we can see which one contributes to tritium production more
tbr_tally.nuclides = [openmc.Nuclide('Li6'), openmc.Nuclide('Li7')] 
my_tallies = openmc.Tallies([tbr_tally])

The next code block runs the simulation.

In [ ]:
# Run OpenMC!
model = openmc.model.Model(my_geometry, my_materials, my_settings, my_tallies)

!rm summary.h5
!rm statepoint.*.h5
sp_filename = model.run()

Finally, this code block loads up the output file and extracts the simulation result.

In [ ]:
# open the results file
sp = openmc.StatePoint(sp_filename)

# access the tally using pandas dataframes
tbr_tally = sp.get_tally(name='TBR')
df = tbr_tally.get_pandas_dataframe()

# prints the contents of the dataframe
df

We can access the values from the dataframe

In [ ]:
# sums up all the values in the mean column
tbr_tally_result = df['mean'].sum()

# sums up all the values in the std. dev. column
tbr_tally_std_dev = df['std. dev.'].sum()

# print the results
print('The tritium breeding ratio was found, TBR = ', tbr_tally_result)
print('Standard deviation on the tbr tally is ', tbr_tally_std_dev)

We can also access the individual results of tritium production in each isotope. This is provided because we added ```.nucl

**Learning Outcomes for Part 1:**

- Running simple neutronics simulations with OpenMC and tallying TBR using a cell tally.

In [ ]:
# gets the row that has li6 in the nuclide column and 
lithium_6_contribution = df[df['nuclide'] == 'Li6']['mean'].sum()

lithium_7_contribution = df[df['nuclide'] == 'Li7']['mean'].sum()

print('The tritium breeding by lithium 6 is ', lithium_6_contribution)
print('The tritium breeding by lithium 7 is ', lithium_7_contribution)

**Learning Outcomes for Part 1:**

- Running simple neutronics simulations with OpenMC and tallying TBR using a cell tally.
- Identify the which lithium isotope makes the majority of tritium production